## Import Libraries

In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

In [2]:
# Variables
season = "2021-22"

## Import files

In [3]:
all_events = pd.read_csv(r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
all_players = pd.read_csv(r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"]) 



## Functions

In [4]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [5]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '1341730'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


In [6]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost',
       'photo', 'PPG', 'second_name', 'selected_by_percent', 'status',
       'team_id', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'thre

In [7]:
#all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,100.0,75.0,80201,0,0,-5,5,0,1,1.0,...,NaN,NaN,1.333333,0.288889,-0.155556,4,ARS,1210,1270,GKP
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
2,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
3,NaN,NaN,225321,0,0,6,-6,3,1,18.0,...,NaN,NaN,5.315789,1.039216,0.647059,4,ARS,1210,1270,GKP
4,NaN,NaN,220682,0,0,-1,1,0,1,1.0,...,NaN,NaN,NaN,0.000000,-0.512821,4,ARS,1210,1270,GKP


In [8]:
my_team

,element,position,multiplier,is_captain,is_vice_captain
0,270,1,2,True,False
1,584,2,1,False,False
2,237,3,1,False,False
3,370,4,1,False,False
4,420,5,1,False,True
5,251,6,1,False,False
6,240,7,1,False,False
7,681,8,1,False,False
8,468,9,1,False,False
9,450,10,1,False,False


In [9]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [10]:
my_team.sort_values(by=['player_type_id' , 'selected_by_percent'])['web_name']



0               de Gea
11            Ramsdale
1            Cucurella
14              Alonso
3             Reguilón
13             Cancelo
2     Alexander-Arnold
5            De Bruyne
7             Coutinho
12                Saka
6                 Jota
4                Bowen
8                 King
10             Watkins
9               Dennis
Name: web_name, dtype: object

In [12]:
my_team.to_csv (r'./data/' + season + '/Team_data/my_team.csv', index = False, header=True)

In [13]:
my_team

,multiplier,is_captain,is_vice_captain,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,2,True,False,NaN,NaN,51940,0,0,3,-3,...,NaN,NaN,4.080000,0.773585,0.396226,4,MUN,1210,1270,GKP
1,1,False,False,NaN,NaN,179268,0,0,1,-1,...,NaN,NaN,3.887370,0.745098,0.352941,3,BHA,1130,1130,DEF
2,1,False,False,100.0,100.0,169187,0,0,10,-10,...,NaN,NaN,7.188940,0.835294,0.600000,5,LIV,1330,1350,DEF
3,1,False,False,100.0,100.0,199249,-1,1,3,-3,...,NaN,NaN,4.864865,0.773585,0.396226,4,TOT,1210,1270,DEF
4,1,False,True,NaN,NaN,178186,1,-1,6,-6,...,NaN,NaN,6.259398,0.830986,0.549296,4,WHU,1200,1200,MID
5,1,False,False,100.0,100.0,61366,-2,2,0,0,...,2.0,NaN,7.385787,0.450000,0.283333,5,MCI,1350,1370,MID
6,1,False,False,25.0,100.0,194634,0,0,10,-10,...,NaN,NaN,6.856106,0.647059,0.411765,5,LIV,1330,1350,MID
7,1,False,False,NaN,NaN,84583,1,-1,3,-3,...,NaN,NaN,9.274809,0.931507,0.657534,3,AVL,1130,1160,MID
8,1,False,False,100.0,100.0,78007,0,0,4,-4,...,3.0,NaN,3.776224,0.576271,0.237288,2,WAT,1050,1050,FWD
9,1,False,False,100.0,100.0,230251,-1,1,11,-11,...,NaN,NaN,5.258673,0.754098,0.426230,2,WAT,1050,1050,FWD
